In [1]:
import os
import librosa
import librosa.display
import numpy as np
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [2]:
import torch
import torchaudio
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import random
#from plot_audio import plot_specgram, plot_waveform
os.getcwd()

/Users/jansta/miniconda3/envs/pytorch2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jansta/miniconda3/envs/pytorch2/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


'/Users/jansta/learn/acoustics'

In [3]:
from audio_ds_model import AudioDataset, AudioClassifNet

In [4]:
dict_mats = np.load('/Users/jansta/learn/acoustics/dict_mats.npy', allow_pickle=True).item()


In [5]:
# t = dict_mats['A']['can_opening'][2]
# print(t.shape)
# t2 = t / t.max()
# print(np.max(t), np.min(t), np.mean(t), np.std(t))
# #plt.plot(t)
# plt.hist(t, bins=100)

In [6]:
len(dict_mats['A']['can_opening'][3])

all_labels = dict_mats['A'].keys()
print(all_labels)

dict_keys(['dog', 'chirping_birds', 'vacuum_cleaner', 'thunderstorm', 'door_wood_knock', 'can_opening', 'crow', 'clapping', 'fireworks', 'chainsaw', 'airplane', 'mouse_click', 'pouring_water', 'train', 'sheep', 'water_drops', 'church_bells', 'clock_alarm', 'keyboard_typing', 'wind', 'footsteps', 'frog', 'cow', 'brushing_teeth', 'car_horn', 'crackling_fire', 'helicopter', 'drinking_sipping', 'rain', 'insects', 'laughing', 'hen', 'engine', 'breathing', 'crying_baby', 'hand_saw', 'coughing', 'glass_breaking', 'snoring', 'toilet_flush', 'pig', 'washing_machine', 'clock_tick', 'sneezing', 'rooster', 'sea_waves', 'siren', 'cat', 'door_wood_creaks', 'crickets'])


In [7]:
chosen_labels = ['crickets', 'can_opening', 'dog','chirping_birds', 'chainsaw', 'crying_baby', 'rooster', 'sea_waves','toilet_flush'][:]

In [8]:
#chosen_labels = list(all_labels)[16:26]
encoded_labels = {}
for i, label in enumerate(chosen_labels):
    encoded_labels[label] = i

In [9]:
transform = transforms.Compose(
    [transforms.Resize((64,431)),
    transforms.Grayscale(num_output_channels=1),
    #transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
    ])

In [10]:
# Create dataset with transform
dataset = AudioDataset(dict_mats['A'], chosen_labels, encoded_labels, transform=transform)
datasetB = AudioDataset(dict_mats['B'], chosen_labels, encoded_labels, transform=transform)

# # Split dataset
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create dataloaders
batch_size = 4
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(datasetB, batch_size=batch_size, shuffle=True)



In [11]:
# Test the dataloader
for i, (inputs, labels) in enumerate(train_loader):
    print(f"Batch {i+1}:")
    print(f"Input batch size: {inputs.size()}")
    print(f"Labels: {labels}")
    print("-" * 30)
    break  # Just to test the first batch

Batch 1:
Input batch size: torch.Size([4, 1, 64, 431])
Labels: tensor([7, 2, 6, 0])
------------------------------


In [12]:


# class AudioClassifNet(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)  
#         self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.flatten = nn.Flatten()
#         #self.fc1 = nn.Linear(64 * 4 * 53, 256)  # Adjusted based on pooling layers
#         self.fc1 = nn.Linear(64 * 4 * 26, 256)  # Adjusted based on pooling layers
#         self.fc2 = nn.Linear(256, 128)
#         self.fc3 = nn.Linear(128, n_classes)  # classes
#         self.relu = nn.ReLU()
#         self.softmax = nn.LogSoftmax(dim=1)
    
#     def forward(self, x):
#         x = self.conv1(x)  # out: (BS, 16, 64, 431)
#         x = self.relu(x)
#         x = self.pool(x)   # out: (BS, 16, 32, 215)
#         x = self.conv2(x)  # out: (BS, 32, 32, 215)
#         x = self.relu(x)
#         x = self.pool(x)   # out: (BS, 32, 16, 107)
#         x = self.conv3(x)  # out: (BS, 64, 8, 107)
#         x = self.relu(x)
#         x = self.pool(x)   # out: (BS, 64, 8, 53)
#         x = self.pool(x)   # additional pooling layer - out: (BS, 64, 4, 26) 
#         x = self.flatten(x) # out: (4, 3328)
#         x = self.fc1(x)  # out: (BS, 256)
#         x = self.relu(x)
#         x = self.fc2(x)  # out: (BS, 128)
#         x = self.relu(x)
#         x = self.fc3(x)  # out: (BS, n_classes)
#         x = self.softmax(x)
#         return x

In [13]:
def check_for_nans(tensor, name):
    if torch.isnan(tensor).any():
        print(f"NaNs found in {name}")
        return True
    return False



In [14]:
## Create an  instance of the model:
n_classes = len(chosen_labels)
model = AudioClassifNet(n_classes)
rate_l = 0.0005

In [15]:
from training_func import run_training

In [16]:
for lr in [0.001, 0.00105][:1]: 
    losses_epoch_mean, acc, cm = run_training(model, train_loader, val_loader, rate_l=lr, NUM_EPOCHS = 500, save=True)

Epoch 0/500, Loss: 2.5567397756708994
Epoch 50/500, Loss: 0.0005706311221285
Epoch 100/500, Loss: 0.0000090919989393
Epoch 150/500, Loss: 0.0000006953815239
Epoch 200/500, Loss: 0.0000000645716455
Epoch 250/500, Loss: 0.0000000049670533
Epoch 300/500, Loss: 0.0000000000000000
Epoch 350/500, Loss: 0.0000000000000000
Epoch 400/500, Loss: 0.0000000000000000
Epoch 450/500, Loss: 0.0000000000000000


NameError: name 'sns' is not defined

In [17]:
# y_val = []
# y_val_hat = []
# for i, data in enumerate(val_loader):
#     inputs, y_val_temp = data
#     with torch.no_grad():
#         y_val_hat_temp = model(inputs).round()
    
#     y_val.extend(y_val_temp.numpy())
#     y_val_hat.extend(y_val_hat_temp.numpy())

In [ ]:

# # Accuracy
# acc = accuracy_score(y_val, np.argmax(y_val_hat, axis=1))
# print(f'Accuracy: {acc*100:.2f} %')
# # confusion matrix
# cm = confusion_matrix(y_val, np.argmax(y_val_hat, axis=1))
sns.heatmap(cm, annot=True, xticklabels=chosen_labels, yticklabels=chosen_labels)


In [19]:
#torch.save(model.state_dict(), f'audio_classification_model10_LR{rate_l}_a{acc*100:.0f}%.pth')